In [1]:
import sys
sys.path.insert(0, "../")
import BatchPaSR as bp
from CanteraTools import *    
import plotly
import plotly.plotly as pplotly
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='edwin217', api_key='hoh1lSzTIVSeckYhrNYL')
import warnings
import IPython.core.formatters
warnings.filterwarnings('ignore', category=UserWarning)



### phi_global = 0.635

### phi_main = 0.3719 

### tau_main = 20 - 0.28 ms

### P = 25 kg

### total_mass = 1 kg

In [2]:
phi_global = 0.635
phi_main = 0.3719
tau_main = 20 - 0.28
tau_global = 20
tau_mix = [0.005, 0.01, 0.05, 0.1, 0.3]
P = 25 * ct.one_atm
miliseconds = 0.001
total_mass = 1
fs = 0.058387057492574147
# Go ahead and run main burner since everything is fixed
[mfm, mam, mfs, mas] = solve_mass_airsplit(phi_global, phi_main, total_mass, 1)
# print(f"mfm = {mfm:.3f}; mam = {mam:.3f}; mfs = {mfs:.3f}; mas = {mas:.3f}")
# print(f"phi_main = {mfm/mam/fs:.3f}; phi_sec = {mfs/mas/fs:.3f}; phi_global = {(mfm+mfs)/(mas+mam)/fs}")
[vit_reactor, main_burner_DF] = run_main_burner(phi_main, tau_main*milliseconds)
vit_particle = bp.Particle.from_gas(vit_reactor.thermo, particle_mass = mfm+mam)
fuel = ct.Solution('gri30.xml')
fuel.TPX = 300, P, {'CH4':1} # TODO: Come up with a more general way of doing secondary gas so that we can have both fuel and air 
air = ct.Solution('gri30.xml'); 
air.TPX = 650, P, {'O2':0.21, 'N2':0.79}
secondary_gas = mix([fuel, air], [mfs, mas], P = P)
secondary_part = bp.Particle.from_gas(secondary_gas, particle_mass=mfs+mas)

Vitiator end time: 19.719906328977743 milliseconds
Initial mainBurnerDF length: 379
New mainBurnerDF length: 1291


# Test 1: Running a LiME with Fixed Entrainment of Both Streams, tau_mix = 0.03 ms

In [3]:
mass_main = mfm + mam 
mass_sec = mfs + mas 
J_ratio = 3
rho_crossflow = vit_reactor.thermo.density
u_crossflow = 20 # m/s
rho_jet = secondary_gas.density
u_jet = ((J_ratio * (rho_crossflow * u_crossflow * u_crossflow))/rho_jet)**0.5
print(f"u_crossflow = {u_crossflow:.2f} m/s")
print(f"u_jet = {u_jet:.2f} m/s\n")
A_crossflow = 1 # m2
A_jet = A_crossflow/10

mdot_main = vit_reactor.thermo.density * A_crossflow * u_crossflow 
mdot_sec = rho_jet * A_jet * u_jet
print(f"Vitiated product mass = {mass_main:.3f} kg\nSecondary fluid mass = {mass_sec:.3f} kg\n")
print(f"Vitiated product density = {vit_reactor.thermo.density:.3f} kg/m3\nSecondary fluid density = {secondary_gas.density:.3f} kg/m3\n")
print(f"mdot_main = {mdot_main:.3f} kg/s\nmdot_sec = {mdot_sec:.3f} kg/s\n")
print(f"Time taken to completely entrain main burner fluid = {mass_main/mdot_main*1e3:.4f} ms")
print(f"Time taken to completely entrain jet fluid = {mass_sec/mdot_sec*1e3:.4f} ms")

u_crossflow = 20.00 m/s
u_jet = 20.61 m/s

Vitiated product mass = 0.985 kg
Secondary fluid mass = 0.015 kg

Vitiated product density = 5.765 kg/m3
Secondary fluid density = 16.292 kg/m3

mdot_main = 115.306 kg/s
mdot_sec = 33.573 kg/s

Time taken to completely entrain main burner fluid = 8.5441 ms
Time taken to completely entrain jet fluid = 0.4412 ms


In [4]:
pasbr = bp.LiME([], N_MAX=200, dt=0.001*milliseconds)

In [5]:
t1 = time.time()
def massflowfcnmain(time):
    if time <= mass_main/mdot_main:
        return mdot_main
    else:
        return 0

def massflowfcnsec(time):
    if time <= mass_sec/mdot_sec:
        return mdot_sec
    else:
        return 0
totalTime = 10*milliseconds

dt_entrain = (mass_main/mdot_main)/30
print(f"dt_entrain = {dt_entrain/1e-3:.4f} ms")
t = np.arange(0, totalTime, pasbr.dt)
pfc_vit = bp.ParticleFlowController(pasbr, vit_reactor.thermo, mass_main, dt_entrain, method=massflowfcnmain)
# pfc_vit.gas()
pfc_sec = bp.ParticleFlowController(pasbr, secondary_gas, mass_sec, dt_entrain, method=massflowfcnsec)
# pfc_sec.gas()
for i in range(0,t.size):
    pfc_vit.entrain(t[i])
    pfc_sec.entrain(t[i])
#     pdb.set_trace()
    pasbr.react()
    pasbr.mix(tau_mix=0.03*milliseconds)
t2 = time.time(); 
print(f"Time taken = {(t2-t1)/60:.2f} minutes")

dt_entrain = 0.2848 ms
Time taken = 6.37 minutes


In [6]:
df = pasbr.get_time_history()
particle_historyList = [p.get_time_history(dataFrame=True) for p in pasbr.particle_list]
for i in range(0, len(particle_historyList)):
    particle_historyList[i].age += totalTime - particle_historyList[i].age.iloc[-1]

In [11]:
# T_traces = [go.Scatter(x = particle_historyList[i]['age'], y = particle_historyList[i]['T'], mode = 'lines', yaxis='y2', name = f"Particle {i} Temp 0.03 ms") for i in range(0, len(particle_historyList))]
# T_fig = go.Figure(data=T_traces)
# T_only_url = pplotly.iplot(T_fig, filename='T_0.001ms', fileopt='append')
# T_only_url

T_traces = [go.Scatter(x = particle_historyList[i]['age']/milliseconds, y = particle_historyList[i]['T'], mode = 'lines', yaxis='y2', name = f"Particle {i} Temp 0.03 ms", line = {'dash':'dot', 'width':2.0}) for i in range(0, len(particle_historyList))]
T_fig = go.Figure(data=T_traces)
pplotly.iplot(T_fig, filename='T_0.001ms', fileopt='append')

PlotlyRequestError: <html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx</center>
</body>
</html>


In [ ]:
Y_NO_trace = go.Scatter(x = df['age']/milliseconds, y = df['Y_NO']*1e6, mode = 'lines', name = 'Y_NO Uncorrected'); 
X_NO_trace = go.Scatter(x = df['age']/milliseconds, y = df['X_NO']*1e6, mode = 'lines', name = 'X_NO Uncorrected'); 
X_NO_corr_trace = go.Scatter(x = df['age']/milliseconds, y = correct_nox(df['X_NO'], df['X_H2O'], df['X_O2']), name  = 'X_NO Corrected')

NO_and_T_traces = T_traces + [Y_NO_trace,  X_NO_trace,  X_NO_corr_trace]

multiAxisLayout = go.Layout(title='NO and Temp', xaxis={'title':"Time (ms)"}, yaxis={'title': "NO (ppm)"}, yaxis2={'title':"Temperature(K)", 'side':"right", 'overlaying':"y"})
NO_fig = go.Figure(data = NO_and_T_traces, layout=multiAxisLayout)

NO_url = pplotly.iplot(NO_fig, filename='NO_0.03ms', fileopt='overwrite')
NO_url

In [ ]:
Y_CO_trace = go.Scatter(x = df['age']/milliseconds, y = df['Y_CO']*1e6, mode = 'lines', name = 'Y_CO Uncorrected'); 
X_CO_trace = go.Scatter(x = df['age']/milliseconds, y = df['X_CO']*1e6, mode = 'lines', name = 'X_CO Uncorrected'); 
X_CO_corr_trace = go.Scatter(x = df['age']/milliseconds, y = correct_nox(df['X_CO'], df['X_H2O'], df['X_O2']), name  = 'X_CO Corrected')

CO_and_T_traces = T_traces + [Y_CO_trace, X_CO_trace,  X_CO_corr_trace]

# multiAxisLayout = go.Layout(title='CO and Temp', yaxis={'title': "CO (ppm)", 'type':'log'}, yaxis2={'title':"Temperature(K)", 'side':"right", 'overlaying':"y"})
multiAxisLayout = go.Layout(title='CO and Temp', 
                            yaxis={'title': "CO (ppm)"}, 
                            yaxis2=dict(overlaying= 'y', 
                                        anchor= 'x', 
                                        side= 'right', 
                                        showgrid= False, 
                                        title= 'Temperature (K)'))
CO_fig = go.Figure(data = CO_and_T_traces, layout=multiAxisLayout)

# po.iplot(CO_fig, filename = 'CO_and_temp', image='png')
CO_url = pplotly.iplot(CO_fig, filename='CO_0.03ms', fileopt='overwrite')
CO_url

In [12]:
X_NO_corr_trace = go.Scatter(x = df['age']/milliseconds, y = correct_nox(df['X_NO'], df['X_H2O'], df['X_O2']), name  = 'X_NO Corrected 0.03ms', mode='lines')
X_CO_corr_trace = go.Scatter(x = df['age']/milliseconds, y = correct_nox(df['X_CO'], df['X_H2O'], df['X_O2']), name  = 'X_CO Corrected 0.03ms', mode='lines')


NO_only_layout = go.Layout(xaxis = {'title': 'Time (ms)'}, yaxis = {'title': 'NO (ppm)'})
NO_only_fig = go.Figure(data=[X_NO_corr_trace], layout=NO_only_layout)
NO_only_url = pplotly.iplot(NO_only_fig, filename='NO_only', fileopt='append')

CO_only_layout = go.Layout(xaxis = {'title': 'Time (ms)'}, yaxis = {'title': 'CO (ppm)'})
CO_only_fig = go.Figure(data = [X_CO_corr_trace], layout=CO_only_layout)
CO_only_url = pplotly.iplot(CO_only_fig, filename='CO_only', fileopt='append')

NO_only_url


In [13]:
CO_only_url